In [1]:
import numpy as np
import pandas as pd
import time
from datetime import datetime, date
from datetime import timedelta
import matplotlib.pyplot as plt
from scipy import stats
import scipy.stats as stats
import akshare as ak
# import tushare as ts
# ts.set_token('你自己的token')
# pro = ts.pro_api()
# import efinance as ef
# import baostock as bs
import tqdm
import scipy
import statsmodels.stats.weightstats as sw #单样本和双样本的z检验都是用该函数

In [2]:
start_date=(date.today() - timedelta(days=3650)).strftime("%Y%m%d")
end_date=time.strftime("%Y%m%d",time.localtime())
# get_zs_price_day_sh('000001',start_date,end_date)
print(start_date,end_date)

20130625 20230623


In [3]:
df_index=ak.stock_index_pe_lg(symbol='沪深300')[['日期','指数']]
df_index

,日期,指数
0,2005-04-08,1003.45
1,2005-04-11,995.42
2,2005-04-12,978.70
3,2005-04-13,1000.90
4,2005-04-14,986.98
...,...,...
4421,2023-06-15,3925.50
4422,2023-06-16,3963.35
4423,2023-06-19,3930.91
4424,2023-06-20,3924.24


In [4]:
N=20
M=2

In [5]:
df_index['mean']=0
df_index['std']=0

In [6]:
# 计算移动平均值
df_index['mean'] = df_index.rolling(20).mean()
# 计算移动标准差
df_index['std'] = df_index.rolling(20).std()
df_index

,日期,指数,mean,std
0,2005-04-08,1003.45,NaN,NaN
1,2005-04-11,995.42,NaN,NaN
2,2005-04-12,978.70,NaN,NaN
3,2005-04-13,1000.90,NaN,NaN
4,2005-04-14,986.98,NaN,NaN
...,...,...,...,...
4421,2023-06-15,3925.50,3856.2015,48.242950
4422,2023-06-16,3963.35,3857.1420,50.199435
4423,2023-06-19,3930.91,3855.2210,46.260957
4424,2023-06-20,3924.24,3855.7735,47.049000


In [7]:
df_index['chance']=0
for i in tqdm.trange(len(df_index)):
    if df_index.iloc[i]['指数']>df_index.iloc[i]['mean']+2*df_index.iloc[i]['std']:
        df_index.loc[i,'chance']=-1
    elif df_index.iloc[i]['指数']<df_index.iloc[i]['mean']-2*df_index.iloc[i]['std']:
        df_index.loc[i,'chance']=1
    else:
        pass

100%|██████████| 4426/4426 [00:03<00:00, 1431.18it/s]


In [8]:
df_index['收益'] = np.nan
df_index['收益率'] = np.nan
# 只有所有日期都存在才可以使用（速度原因）
df_index['日均收益率'] = np.nan
# 只有所有日期都存在才可以使用（速度原因）
for i in tqdm.trange(len(df_index)-1):
    # 判断是否缺失
    if df_index['chance'].iloc[i]==0:
        pass
    # chance为1的时机
    elif df_index['chance'].iloc[i]==1:
        for j in range(i+1,len(df_index)):
            if df_index['chance'].iloc[j]==-1:
                df_index['收益'].iloc[i]=df_index['指数'].iloc[j]-df_index['指数'].iloc[i]
                df_index['收益率'].iloc[i]=100*(df_index['指数'].iloc[j]-df_index['指数'].iloc[i])/df_index['指数'].iloc[i]
                df_index['日均收益率'].iloc[i]=100*((df_index['指数'].iloc[j]-df_index['指数'].iloc[i])/df_index['指数'].iloc[i])/(j-i)
                break
            else:
                pass
    # chance为-1
    elif df_index['chance'].iloc[i]==-1:
        for j in range(i+1,len(df_index)):
            if df_index['chance'].iloc[j]==1:
                df_index['收益'].iloc[i]=df_index['指数'].iloc[j]-df_index['指数'].iloc[i]
                df_index['收益率'].iloc[i]=100*(df_index['指数'].iloc[j]-df_index['指数'].iloc[i])/df_index['指数'].iloc[i]
                df_index['日均收益率'].iloc[i]=100*((df_index['指数'].iloc[j]-df_index['指数'].iloc[i])/df_index['指数'].iloc[i])/(j-i)
                break
            else:
                pass
df_index

  0%|          | 0/4425 [00:00<?, ?it/s]D:\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
100%|██████████| 4425/4425 [00:00<00:00, 9184.31it/s]


,日期,指数,mean,std,chance,收益,收益率,日均收益率
0,2005-04-08,1003.45,NaN,NaN,0,NaN,NaN,NaN
1,2005-04-11,995.42,NaN,NaN,0,NaN,NaN,NaN
2,2005-04-12,978.70,NaN,NaN,0,NaN,NaN,NaN
3,2005-04-13,1000.90,NaN,NaN,0,NaN,NaN,NaN
4,2005-04-14,986.98,NaN,NaN,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
4421,2023-06-15,3925.50,3856.2015,48.242950,0,NaN,NaN,NaN
4422,2023-06-16,3963.35,3857.1420,50.199435,-1,NaN,NaN,NaN
4423,2023-06-19,3930.91,3855.2210,46.260957,0,NaN,NaN,NaN
4424,2023-06-20,3924.24,3855.7735,47.049000,0,NaN,NaN,NaN


In [9]:
df_index_chance=df_index[(df_index['日均收益率'] >-100)]
df_index_chance

,日期,指数,mean,std,chance,收益,收益率,日均收益率
34,2005-06-02,818.38,875.5640,26.839650,1,75.63,9.241428,0.236960
57,2005-07-05,849.68,888.6625,18.958400,1,44.33,5.217258,0.326079
58,2005-07-06,842.56,885.5020,21.103750,1,51.45,6.106390,0.407093
60,2005-07-08,829.49,878.8550,24.668002,1,64.52,7.778273,0.598329
73,2005-07-27,894.01,850.7795,17.129490,-1,10.20,1.140927,0.025930
...,...,...,...,...,...,...,...,...
4372,2023-04-03,4090.57,4006.6705,38.913015,-1,-127.90,-3.126704,-0.208447
4373,2023-04-04,4103.11,4009.3835,43.615842,-1,-140.44,-3.422770,-0.244484
4387,2023-04-25,3962.67,4074.9405,55.479681,1,0.68,0.017160,0.000490
4405,2023-05-24,3859.09,3978.2250,45.957186,1,104.26,2.701673,0.158922


In [10]:
A=[]
B=[]
df_index_time=df_index_chance[df_index_chance['日期']>=datetime.strptime('2020-01-01', "%Y-%m-%d").date()]
# df_index_time=df_index_chance[df_index_chance['日期']>='2020-01-01']
for i in tqdm.trange(len(df_index_time)-1):
    if df_index_time.iloc[i]['chance']==1:
        A.append(df_index_time.iloc[i]['日均收益率'])
    elif df_index_time.iloc[i]['chance']==-1:
        B.append(df_index_time.iloc[i]['日均收益率'])

100%|██████████| 87/87 [00:00<00:00, 3345.60it/s]


In [11]:
print(len(A),len(B))

45 42


In [12]:
# stats.normaltest
statistic, p_value = scipy.stats.normaltest (A, axis=0)
print('A - 统计量：', statistic)
print('A - P值：', p_value)
if p_value>0.05:
    print('A服从正态分布')
else:
    print('A不服从正态分布')

statistic, p_value = scipy.stats.normaltest (B, axis=0)
print('B - 统计量：', statistic)
print('B - P值：', p_value)
if p_value>0.05:
    print('B服从正态分布')
else:
    print('B不服从正态分布')

A - 统计量： 3.6943713255403305
A - P值： 0.15768030800728933
A服从正态分布
B - 统计量： 23.645829633826352
B - P值： 7.33454766050903e-06
B不服从正态分布


In [13]:
# z,p=sw.ztest(A,B,value=0,alternative="larger")#{“two-sided”, larger, smaller}
z,p=sw.ztest(A,value=0,alternative="larger")#{“two-sided”, larger, smaller}
print("z值={},p值={}".format(z,p))

z值=3.870538128185361,p值=5.4297684955175646e-05


In [14]:
# t_statistic, p_value = stats.ttest_ind(A, B)
t_statistic, p_value = stats.ttest_1samp(a=A, popmean=0, alternative='greater') #'less', 'greater' or 'two-sided'
print('T统计量：', t_statistic)
print('P值：', p_value)
if p_value>0.05:
    print('无显著差异')
else:
    print('有显著差异')

T统计量： 3.870538128185361
P值： 0.0001779016442811913
有显著差异


In [15]:
# 如果A和B不一样长度，需要剔除末尾的数据
# A_1=A[:-3]
# B_1=B[:-1]
# print(len(A_1),len(B))
# print(len(A),len(B_1))

In [16]:
# scipy.stats.wilcoxon( A_1, B, correction = False, alternative = 'greater' ) #{“two-sided”, “greater”, “less”}

In [17]:
# print(A)
# print(B)

BOLL策略可以获得收益。